# Residency Day Project
## 
### Name: Roshan Acharya, Avinna Bhattarai
### Course: # MSCS 634: Data Science and Machine Learning


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

### Set style for better plots

In [7]:
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## Step 1: Load and Prepare the Dataset

In [8]:
# Load the dataset
df = pd.read_csv('Data/mobile.csv')
# Display the first five rows
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'Data/mobile.csv'

**Box Plot: Price Distribution by Launch Status**

The earlier box plot was visually misleading due to axis mislabeling and an overwhelming presence of outliers. Here, we address those issues by:

Correcting axis labels.
Applying a clear color palette.
Limiting the y-axis range to enhance visibility.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='tag', y='price', palette='Set2')

# Set appropriate axis labels and title
plt.title('Price Distribution by Launch Status', fontsize=16)
plt.xlabel('Launch Status', fontsize=12)
plt.ylabel('Price (INR)', fontsize=12)

# Limit y-axis to minimize visual distortion from extreme outliers
plt.ylim(0, 150000)

plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


**Interpretation**

Launched phones tend to be priced lower on average than Upcoming or Rumored models.
Rumored phones exhibit the widest price spread, possibly reflecting speculation around premium releases.
Outlier clipping at ₹150,000 improves readability by removing distortion caused by a few flagship models priced upwards of ₹400,000.